In [1]:
import pandas as pd
import numpy as np
from some_functions import get_db

In [2]:
readers = pd.read_csv(r"C:\Users\a814811\OneDrive - Atos\RecommenderSystem\readers.csv")
readers = readers.rename(columns={"id":"user_id", "art_id":"nzz_id"})
art_db = get_db(r'C:\Users\a814811\OneDrive - Atos\RecommenderSystem\art_clean_wt_all_popularity.csv')
art_db = art_db.loc[:,['nzz_id','author','department','popularity']] #skrócenie do potrzebnych rzeczy

In [3]:
read_counts = readers["user_id"].value_counts(sort=True)
read_counts = read_counts.rename_axis("user_id").reset_index(name="read_count")

# Biorę pod uwagę tylko użytkowników, którzy przeczytali minimum 5 artykułów
min_read_count = 3
read_counts = read_counts[read_counts["read_count"] > min_read_count]

readers = readers[readers["user_id"].isin(read_counts["user_id"])]

In [4]:
# Train/Test split
from sklearn.model_selection import train_test_split

random_state = None
readers_train, readers_test = train_test_split(readers,
                                   stratify=readers["user_id"], 
                                   test_size=0.20,
                                   random_state=random_state)

In [5]:

from popularity_model import *
from model_evaluator import ModelEvaluator
model_evaluator = ModelEvaluator(k_list = [5, 10, 15])

### sprawdzam submodele i główny
#### dla authora są mocno zaniżone wyniki, bo często nie rekomenduje nic a test tego nie uwzględnia, podobnie dla department (w mniejszym stopniu)
#### merged i popularity działają normalnie (zawsze coś jest rekomednowane)

In [6]:
# merged model
p_model = Popularity_model_merge(art_db,readers)
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(p_model, readers, readers_train, readers_test)
print(f'\nGlobal metrics:\n{cf_global_metrics}')

999 users processed

Global metrics:
{'modelName': 'p_model', 'recall@5': 0.22400317460317462, 'precision@5': 0.04480063492063492, 'f1_score@5': 0.07466772486772487, 'ndcg@5': 0.1561412944064938, 'recall@10': 0.35361746031746033, 'precision@10': 0.035361746031746036, 'f1_score@10': 0.0642940836940837, 'ndcg@10': 0.19743235974972606, 'recall@15': 0.4404186507936507, 'precision@15': 0.029361243386243387, 'f1_score@15': 0.055052331349206354, 'ndcg@15': 0.21851573866821797}


In [89]:
# x = ['f1_score@5','f1_score@10','f1_score@15']
# for it in x: print(f'{it}: {cf_global_metrics[it]}')
results = pd.DataFrame([],
        columns=['modelName',
        'recall@5',
        ' precision@5',
        'f1_score@5',
        'ndcg@5',
        'recall@10',
        'precision@10',
        'f1_score@10',
        'ndcg@10',
        'recall@15',
        'precision@15',
        'f1_score@15',
        'ndcg@15',
        'weight'])
v = list(cf_global_metrics.values())
v.append((1,2,3))
r1 = pd.DataFrame([v],
        columns=['modelName',
        'recall@5',
        ' precision@5',
        'f1_score@5',
        'ndcg@5',
        'recall@10',
        'precision@10',
        'f1_score@10',
        'ndcg@10',
        'recall@15',
        'precision@15',
        'f1_score@15',
        'ndcg@15',
        'weight'])
results = results.append(r1,ignore_index=True)

results.to_csv("res.csv", encoding="utf-8", index=False)

## Wizualizacja